In [ ]:
import urllib.request
import json
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
# Make the request to the API
req = urllib.request.Request('https://livescore-api.com/api-client/scores/history.json?secret=dRjupfSdWvLvhbBgdglu3iY0GWZYVYF8&key=Z1F4hMWtXoYZiZYG&package_id=4&page=8000')
response = urllib.request.urlopen(req)
# Read the response and convert it from JSON to a Python dictionary
data = json.loads(response.read())
# Assuming the relevant data is in a key named 'data' (you will need to adjust this based on the actual structure of the JSON response)
# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame(data['data'])

In [ ]:
all_data = pd.DataFrame()
# Loop through pages 4000 to 14000
for page in range(14300, 14604):  # Adjust the range as needed
    print(f"Processing page: {page}")
    # Construct the request URL for the current page
    url = f'https://livescore-api.com/api-client/scores/history.json?secret=dRjupfSdWvLvhbBgdglu3iY0GWZYVYF8&key=Z1F4hMWtXoYZiZYG&package_id=4&page={page}'
    # Make the request to the API
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    # Read the response and convert it from JSON to a Python dictionary
    data = json.loads(response.read())
    df = pd.DataFrame(data['data'])
    df_match = pd.json_normalize(df['match'])
    all_data = pd.concat([all_data, df_match], ignore_index=True)
# Display the shape of the DataFrame to see how many rows of data were collected
all_data.shape()

In [ ]:
selected_columns = ['home_name', 'score', 'ht_score', 'away_name', 'id']
df_selected = all_data[selected_columns]
df_selected.to_csv('history_data.csv', index=False)

In [ ]:
# Initialize an empty list to store match statistics
match_stats_list = []

# Loop through each match id in the sliced DataFrame
for index, row in df_selected.iterrows():
    print(f"Processing id: {row['id']}")
    match_id = row['id']
    url = f'https://livescore-api.com/api-client/matches/stats.json?match_id={match_id}&key=Z1F4hMWtXoYZiZYG&secret=dRjupfSdWvLvhbBgdglu3iY0GWZYVYF8'

    # Make the request to the API
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    data = json.loads(response.read())

    # Prepare a dictionary to hold the match statistics
    stats = data['data']
    match_dict = {
        'home_name': row['home_name'],
        'score': row['score'],
        'ht_score': row['ht_score'],
        'away_name': row['away_name'],
        'id': row['id']
    }

    # Loop through each statistic and split the values for home and away teams
    if isinstance(stats, dict):
        for stat, value in stats.items():
            if value is not None and ':' in value:
                home_value, away_value = value.split(':')
                match_dict[f'{stat}_home'] = home_value
                match_dict[f'{stat}_away'] = away_value
            else:
                match_dict[stat] = value  # For stats not split between home and away

    match_stats_list.append(match_dict)

# Convert the list of dictionaries to a DataFrame
df_match_stats = pd.DataFrame(match_stats_list)
df_match_stats

df_match_stats.to_csv('match_stats.csv', index=False)

In [2]:
import pandas as pd
df_match_stats=pd.read_csv('/workspaces/match_football_prediction/dangdienra/dulieu/match_stats.csv')

In [3]:
# Danh sách các cột ban đầu
cols = df_match_stats.columns
# Lọc ra các cột có chứa "_away", "_home", và các cột cần giữ
cols_with_home_away = [col for col in cols if "_away" in col or "_home" in col or col in ["home_name", "score","ht_score","away_name", "id"]]

# Tạo dataframe mới với các cột đã lọc
df = df_match_stats[cols_with_home_away]
df.fillna(0)


,home_name,score,ht_score,away_name,id,yellow_cards_home,yellow_cards_away,red_cards_home,red_cards_away,possesion_home,...,throw_ins_home,throw_ins_away,penalties_home,penalties_away,substitutions_home,substitutions_away,goal_kicks_home,goal_kicks_away,treatments_home,treatments_away
0,FC Sevastopol,1 - 0,0 - 0,Legion-Dynamo,519024,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Biolog Novokubansk,3 - 2,2 - 1,FC Spartak Nalchik,519025,5.0,5.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Backa Topola,3 - 2,2 - 1,Vojvodina,519031,3.0,2.0,0.0,0.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Komarno,2 - 0,1 - 0,FC STK 1914 Samorin,519032,0.0,1.0,0.0,0.0,56.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Radomlje,1 - 2,1 - 1,NS Mura,519033,1.0,3.0,0.0,0.0,56.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7179,Santa Fe,1 - 0,0 - 0,CD Once Caldas,530952,1.0,1.0,0.0,0.0,56.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7180,Cruz Azul,1 - 1,1 - 1,CF America,530953,0.0,0.0,0.0,0.0,62.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7181,Sydney II,1 - 3,1 - 1,North West Sydney Spirit FC,530955,3.0,1.0,0.0,0.0,44.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7182,Bangladesh Police,1 - 2,0 - 1,Fortis,530956,1.0,1.0,0.0,0.0,50.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
import numpy as np
exclude_columns = ['home_name', 'score', 'ht_score', 'away_name', 'id']

# Columns to check for zeros
check_columns = df.columns.difference(exclude_columns)

# Calculate the sum across the check_columns for each row
# Replace 0.0 with np.nan and then use dropna to remove rows with all zeros in check_columns
df = df.replace({col: {0.0: np.nan} for col in check_columns}).dropna(how='all', subset=check_columns)

# Replace np.nan back to 0.0 if needed
df.fillna(0.0, inplace=True)

In [5]:
exclude_columns = ['home_name', 'score', 'ht_score', 'away_name', 'id']

# Columns to check for zeros
check_columns = df.columns.difference(exclude_columns)

# Calculate the sum across the check_columns for each row
# Replace 0.0 with np.nan and then use dropna to remove rows with all zeros in check_columns
df = df.replace({col: {0.0: np.nan} for col in check_columns}).dropna(how='all', subset=check_columns)

# Replace np.nan back to 0.0 if needed
df.fillna(0.0, inplace=True)
# Tách cột 'score' thành 'score_home' và 'score_away'
df[['score_home', 'score_away']] = df['score'].str.split(' - ', expand=True)
df[['ht_score_home', 'ht_score_away']] = df['ht_score'].str.split(' - ', expand=True)

# Chuyển đổi kiểu dữ liệu của 'score_home' và 'score_away' thành số nguyên
df['score_home'] = df['score_home'].astype(int)
df['score_away'] = df['score_away'].astype(int)

# Xóa cột 'score'
df = df.drop(columns=['score'])
df = df.drop(columns=['ht_score'])
df = df.drop(columns=['id'])
columns_to_exclude = ['home_name', 'away_name', 'ht_score_home', 'ht_score_away','score_home', 'score_away']
columns_to_divide = df.columns.difference(columns_to_exclude)
df[columns_to_divide] = df[columns_to_divide] / 2
df

,home_name,away_name,yellow_cards_home,yellow_cards_away,red_cards_home,red_cards_away,possesion_home,possesion_away,corners_home,corners_away,...,substitutions_home,substitutions_away,goal_kicks_home,goal_kicks_away,treatments_home,treatments_away,score_home,score_away,ht_score_home,ht_score_away
0,FC Sevastopol,Legion-Dynamo,0.0,0.5,0.0,0.0,0.0,0.0,4.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0
1,Biolog Novokubansk,FC Spartak Nalchik,2.5,2.5,0.0,0.5,0.0,0.0,4.0,1.5,...,0.0,0.0,0.0,0.0,0.0,0.0,3,2,2,1
2,Backa Topola,Vojvodina,1.5,1.0,0.0,0.0,25.0,25.0,2.0,2.5,...,0.0,0.0,0.0,0.0,0.0,0.0,3,2,2,1
3,Komarno,FC STK 1914 Samorin,0.0,0.5,0.0,0.0,28.0,22.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,0,1,0
4,Radomlje,NS Mura,0.5,1.5,0.0,0.0,28.0,21.5,1.5,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7179,Santa Fe,CD Once Caldas,0.5,0.5,0.0,0.0,28.0,22.0,1.5,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0
7180,Cruz Azul,CF America,0.0,0.0,0.0,0.0,31.0,19.0,3.5,1.5,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1
7181,Sydney II,North West Sydney Spirit FC,1.5,0.5,0.0,0.0,22.0,28.0,2.5,2.5,...,0.0,0.0,0.0,0.0,0.0,0.0,1,3,1,1
7182,Bangladesh Police,Fortis,0.5,0.5,0.0,0.0,25.0,25.0,2.5,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,2,0,1


In [6]:
df.to_csv('data_processed.csv', index=False)  